In [4]:
from gensim.models import KeyedVectors

# Load the model
model = KeyedVectors.load_word2vec_format('C:/Users/mouni/Downloads/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec')

# Now you can use the model
vector = model['word']

In [5]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer

from utils import (cosine_similarity, get_dict,
                   process_tweet)
from os import getcwd

In [6]:
print(f"Vocabulary size: {len(model.key_to_index)}")
print(f"Vector dimension: {model.vector_size}")
print(f"Total size: {model.vectors.shape}")

# Example of getting a vector for a word
vector = model['word']
print(f"Vector for 'word': {vector}")
print(f"Length of vector: {len(vector)}")

Vocabulary size: 999994
Vector dimension: 300
Total size: (999994, 300)
Vector for 'word': [-0.0078  0.0075  0.0203  0.0991  0.0315 -0.0284  0.144   0.1352  0.0612
  0.0649 -0.0026 -0.0589  0.0478 -0.0337 -0.0233  0.0469 -0.0836 -0.0053
 -0.0024  0.0059 -0.097  -0.0849 -0.1521  0.0512 -0.0095 -0.0535  0.0353
  0.101  -0.0254  0.0507  0.1445  0.0814  0.0186 -0.0705 -0.005   0.1171
 -0.0342 -0.0678  0.0331  0.124   0.0532  0.1233 -0.0551 -0.0339  0.0393
  0.042   0.0718  0.0245  0.0014  0.0358  0.0248  0.0603 -0.629   0.0427
 -0.1074 -0.0039  0.021  -0.2132  0.0053 -0.048   0.1041  0.0203 -0.1299
  0.0821  0.0068 -0.1315  0.1093  0.0268  0.0453  0.086   0.0879 -0.081
 -0.0609 -0.0699  0.0355  0.1278  0.0056 -0.1478 -0.0338  0.053   0.042
  0.0057 -0.1486 -0.254  -0.0338  0.031   0.1076  0.1301  0.2479 -0.0424
  0.0194  0.0241  0.0553 -0.021   0.0199  0.1346 -0.0271 -0.0165  0.0626
 -0.0277 -0.1648  0.1067  0.0814  0.0561  0.02    0.0851  0.077   0.1487
  0.0184  0.0639  0.2449  0.0458  0

In [7]:
import re

# Updated preprocessing function to remove unwanted words
def preprocess_word(word):
    # Remove stock market tickers like $GE
    if re.search(r'\$\w*', word):
        return None
    # Remove old style retweet text "RT"
    if re.search(r'^RT[\s]+', word):
        return None
    # Remove hyperlinks and "www"
    if re.search(r'https?:\/\/.*[\r\n]*', word) or 'www' in word:
        return None
    # Remove hashtags (only removing the hash # sign from the word)
    if re.search(r'#', word):
        return None
    # Remove words containing special characters
    if re.search(r'[^a-zA-Z0-9]', word):
        return None
    return word

# Get the vocabulary of the embeddings
vocab = list(model.index_to_key)

# Apply preprocessing to each word in the vocabulary and store their original vectors
preprocessed_embeddings = {}
for word in vocab:
    preprocessed_word = preprocess_word(word)
    if preprocessed_word:  # Only add non-None words
        preprocessed_embeddings[preprocessed_word] = model[word]

In [8]:
print(f"Vocabulary size: {len(preprocessed_embeddings)}")


Vocabulary size: 763569


In [9]:
import heapq
# Define the Manhattan distance function
def manhattan_distance(v1, v2):
    return np.sum(np.abs(v1 - v2))


# def euclidean_distance(v1, v2):
#     return np.linalg.norm(v1 - v2)

# Define the Cosine similarity function
#nelghouuuh sahbi
# def cosine_similarity(v1, v2):
#     dot_product = np.dot(v1, v2)
#     norm_v1 = np.linalg.norm(v1)
#     norm_v2 = np.linalg.norm(v2)
#     return dot_product / (norm_v1 * norm_v2)
# Function to get top n similar words using Manhattan distance



def get_top_n_similar_words_manhattan(input_vector, embeddings, n=3):
    distances = []

    for word, word_vector in embeddings.items():
        distance = manhattan_distance(input_vector, word_vector)
        distances.append((distance, word))
    
    top_n_words = heapq.nsmallest(n, distances, key=lambda x: x[0])  # Smallest distances for most similar
    return [word for _, word in top_n_words]

# Example input vector (this should be replaced with an actual word vector)
input_word = "fool"
input_vector = model[input_word]

# Get top 8 similar words using Manhattan distance
if(input_word in preprocessed_embeddings):
        top_n_words_manhattan = get_top_n_similar_words_manhattan(input_vector, preprocessed_embeddings,n=7)

# Extract the 7th and 8th most similar words
# if len(top_n_words_manhattan) >= 8:
#     seventh_eighth_words_manhattan = top_n_words_manhattan[0:7]
# else:
#     seventh_eighth_words_manhattan = top_n_words_manhattan[6:]  # In case there are fewer than 8 words

print("7th and 8th synonyms using Manhattan distance:", top_n_words_manhattan)


7th and 8th synonyms using Manhattan distance: ['fool', 'fools', 'idiot', 'stupid', 'foolish', 'deluded', 'misleader']


euclidean_distance:
['happy', 'unhappy', 'happier', 'contented', 'glad', 'pleased', 'sad']
['angry', 'enraged', 'frustrated', 'irate', 'annoyed', 'indignant', 'outraged']
['sad', 'pathetic', 'depressing', 'tragic', 'unhappy', 'sorrowful', 'unfortunate']
['wisdom', 'knowledge', 'wise', 'Wisdom', 'insight', 'sensibleness', 'virtue']
['fool', 'fools', 'idiot', 'misleader', 'trick', 'dumbshit', 'knowitall']
['skillful', 'skilful', 'skilled', 'deft', 'crafty', 'masterful', 'adept']


Manhattan distance:
['happy',        'unhappy', 'happier', 'glad', 'contented', 'grateful', 'hapy']
['angry',       'enraged', 'frustrated', 'irate', 'annoyed', 'furious', 'angered']
['sad', '            pathetic', 'tragic', 'depressing', 'unhappy', 'sorrowful', 'happy']
['wisdom',       'wise', 'knowledge', 'virtue', 'sensibleness', 'Wisdom', 'wiseness']
 ['fool',      'fools', 'idiot', 'stupid', 'foolish', 'deluded', 'misleader']
 ['skillful', 'skilful', 'skilled', 'deft', 'crafty', 'adroit', 'masterful']


 Top 7 synonyms for 'skillful' using Manhattan distance and LSH: ['skillful', 'skilful', 'masterful', 'clever', 'adept', 'skillfully', 'perceptive']
Top 7 synonyms for 'fool' using Manhattan distance and LSH: ['fool', 'fools', 'nobhead', 'diguise', 'knowitall', 'dummie', 'suckup']
Top 7 synonyms for 'wisdom' using Manhattan distance and LSH: ['wisdom', 'wise', 'sagacity', 'solicitousness', 'stumblings', 'grandure', 'scrutable']
Top 7 synonyms for 'sad' using Manhattan distance and LSH: ['sad', 'horrible', 'strange', 'troubling', 'scary', 'obessed', 'unsettling']
Top 7 synonyms for 'angry' using Manhattan distance and LSH: ['angry', 'indignant', 'vengeful', 'distrubed', 'strident', 'disturbed', 'wose']
Top 7 synonyms for 'happy' using Manhattan distance and LSH: ['happy', 'happier', 'embarrassed', 'Liselott', 'Selonians', 'germanised', 'Ebizur']


Top 4 synonyms for 'skillful' using Manhattan distance and LSH:
['deft', 'crafty', 'adroit', 'masterful']
Top 4 synonyms for 'fool' using Manhattan distance and LSH:
['idiot', 'foolish', 'daftie', 'overanalyse']
Top 4 synonyms for 'wisdom' using Manhattan distance and LSH: 
['knowledge', 'virtue', 'foresightedness', 'staidness']
Top 4 synonyms for 'sad' using Manhattan distance and LSH: 
['tragic', 'depressing', 'unhappy', 'unforunate']
Top 4 synonyms for 'angry' using Manhattan distance and LSH: 
['enraged', 'frustrated', 'irate', 'furious']




In [10]:
import numpy as np

class LSH:
    def __init__(self, num_planes, input_dim, num_universes):
        self.num_planes = num_planes
        self.input_dim = input_dim
        self.num_universes = num_universes
        
        np.random.seed(0)
        self.planes_l = [np.random.normal(size=(input_dim, num_planes)) for _ in range(num_universes)]
        
        self.hash_tables = []
        self.id_tables = []
        for _ in range(num_universes):
            hash_table = {i: [] for i in range(2 ** num_planes)}
            id_table = {i: [] for i in range(2 ** num_planes)}
            self.hash_tables.append(hash_table)
            self.id_tables.append(id_table)

    def hash_value_of_vector(self, v, planes):
        dot_product = np.dot(v, planes)
        sign_of_dot_product = np.sign(dot_product)
        h = sign_of_dot_product >= 0
        h = np.squeeze(h)
        hash_value = 0
        n_planes = planes.shape[1]
        for i in range(n_planes):
            hash_value += np.power(2, i) * h[i]
        hash_value = int(hash_value)
        return hash_value

    def add_vector(self, vector, label):
        for i in range(self.num_universes):
            planes = self.planes_l[i]
            hash_value = self.hash_value_of_vector(vector, planes)
            self.hash_tables[i][hash_value].append(vector)
            self.id_tables[i][hash_value].append(label)

    def query(self, vector):
        candidates = set()
        for i in range(self.num_universes):
            planes = self.planes_l[i]
            hash_value = self.hash_value_of_vector(vector, planes)
            if hash_value in self.id_tables[i]:
                candidates.update(self.id_tables[i][hash_value])
        return list(candidates)

# Example function to compute Manhattan distance and find top N similar words
def get_top_n_similar_words_manhattan(input_vector, candidate_embeddings, n=7):
    distances = {}
    for word, vector in candidate_embeddings.items():
        distances[word] = np.sum(np.abs(input_vector - vector))
    sorted_words = sorted(distances, key=distances.get)[:n]
    return sorted_words

# Initialize LSH
input_dim = model.vector_size
num_planes = 10
num_universes = 25
lsh = LSH(num_planes, input_dim, num_universes)



Top 7 synonyms for 'skillful' using Manhattan distance and LSH: ['skillful', 'skilful', 'skilled', 'deft', 'crafty', 'adroit', 'masterful']
Top 7 synonyms for 'fool' using Manhattan distance and LSH: ['fool', 'idiot', 'foolish', 'daftie', 'knowitall', 'overanalyse', 'trick']
Top 7 synonyms for 'wisdom' using Manhattan distance and LSH: ['wisdom', 'knowledge', 'virtue', 'foolishness', 'foresightedness', 'staidness', 'infallability']
Top 7 synonyms for 'sad' using Manhattan distance and LSH: ['sad', 'tragic', 'depressing', 'unhappy', 'unfortunate', 'funny', 'unforunate']
Top 7 synonyms for 'angry' using Manhattan distance and LSH: ['angry', 'enraged', 'frustrated', 'irate', 'annoyed', 'furious', 'angered']
Top 7 synonyms for 'happy' using Manhattan distance and LSH: ['happy', 'unhappy', 'happier', 'glad', 'contented', 'hapy', 'happpy']


In [ ]:
# Add vectors to LSH
for word, vector in preprocessed_embeddings.items():
    lsh.add_vector(vector, word)

# Example input list
input_list1 = [
    'skillful',
    'fool',   
    'wisdom',
    'sad',
    'angry',
    'happy',    
]

# Process each word in the input list
for input_word in input_list1:
    if input_word in preprocessed_embeddings:
        input_vector = preprocessed_embeddings[input_word]
        candidate_words = lsh.query(input_vector)
        candidate_embeddings = {word: preprocessed_embeddings[word] for word in candidate_words}
        top_n_words_manhattan = get_top_n_similar_words_manhattan(input_vector, candidate_embeddings, n=7)
        print(f"Top 7 synonyms for '{input_word}' using Manhattan distance and LSH:", top_n_words_manhattan)
    else:
        print(f"Word '{input_word}' not found in the preprocessed embeddings.")


In [12]:
def display_dimensions(self):
        print(f"Number of hyperplanes (num_planes): {self.num_planes}")
        print(f"Input vector dimension (input_dim): {self.input_dim}")
        print(f"Number of universes (num_universes): {self.num_universes}")
        print(f"Shape of planes in each universe: {self.planes_l[0].shape}")
        print(f"Number of buckets in each hash table: {2 ** self.num_planes}")
        print(f"Number of hash tables: {len(self.hash_tables)}")
        for i, (hash_table, id_table) in enumerate(zip(self.hash_tables, self.id_tables)):
            print(f"Universe {i+1}:")
            print(f"  Number of entries in hash table: {sum(len(bucket) for bucket in hash_table.values())}")
            print(f"  Number of entries in ID table: {sum(len(bucket) for bucket in id_table.values())}")
        

In [11]:
lsh.display_dimensions()


AttributeError: 'LSH' object has no attribute 'display_dimensions'